In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import warnings
warnings.simplefilter("ignore")


In [64]:
# color map の作成方法
# https://plotly.com/python/v3/matplotlib-colorscales/
# import parula as par
import matplotlib
from matplotlib import cm
import numpy as np

seismic_cmap = matplotlib.cm.get_cmap('seismic')
_rgb = []
norm = matplotlib.colors.Normalize(vmin=0, vmax=255)
for i in range(0, 255):
    k= matplotlib.colors.colorConverter.to_rgb(seismic_cmap(norm(i)))
    _rgb.append(k)

def matplotlib_to_plotly(cmap, pl_entries):
    
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    for k in range(pl_entries):
        
        list_color = np.array(cmap(k*h)[:3])*255
        C = [ np.uint8(c) for  c in list_color]
#         C = map(np.uint8, np.array(cmap(k*h)[:3])*255)
#         print(k, list_color,C)
#         sys.exit()
        
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])
    return pl_colorscale
# seismic_cmap(0)
# seismic

In [68]:

def load_data(verbose=False):
    path = 'https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv'
    df = pd.read_csv(path)
    if verbose:
        print(df.columns)
        print("data-size = > ",df.shape)
    df = df[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23']]
    return df

def plot_surface(data,vmin,vmax,title = "title"):
    # 3次元データの詳細　https://plotly.com/python/reference/surface/
    
    seismic = matplotlib_to_plotly(seismic_cmap, 255)
    fig = go.Figure(data=[go.Surface(z=data)])
    fig.update_traces(colorscale=seismic,cmid=0, cmin=vmin,cmax=vmax)
    fig.update_layout(title=title, autosize=False,
                      width=800, height=600,
                      
#                   margin=dict(l=65, r=50, b=65, t=90)
                    margin=dict(l=0, r=0, b=10, t=50)
                      
                     )
    fig.show()
    return

In [148]:
def set_P(_point):
#     _x,_y = np.meshgrid(np.arange(-100,100),np.arange(-100,100))
    _x,_y = np.meshgrid(np.arange(200),np.arange(200))
    data = np.zeros((200,200))
    
    n_point = len(_point)
    for i,p in enumerate(_point):
        x,y,Q = p
        x +=100
        y +=100
        data[x,y] = Q
    return data, f"Only Q[C] - N={n_point}"

def set_V(_point):
#     _x,_y = np.meshgrid(np.arange(-100,100),np.arange(-100,100))
    _x,_y = np.meshgrid(np.arange(200),np.arange(200))
    
    _x -=100
    _y -=100 
    _y = np.flipud(_y)
    print(_x[0,0],_x[-1,-1])
    print(_y[0,0],_y[-1,-1])
#     print(_y[:20,:20])
#     sys.exit()
    
    data = np.zeros((200,200))
    K=10
    n_point = len(_point)
    
    vmax,vmin = 3*n_point, -3*n_point
    for i,p in enumerate(_point):
        x,y,Q = p
        
        v_sub = K*Q / np.sqrt((_x-x)**2 + (_y-y)**2)
        v_sub = np.where(v_sub>vmax, vmax,v_sub) #異常値処理
        v_sub = np.where(v_sub<vmin, vmin,v_sub) #異常値処理
        
        data +=v_sub
#         x +=100
#         y +=100
#         data[x,y] = Q
    return data,f"Volt[V] - N={n_point}"


In [149]:
def make_multi_point():
    _point=[]
    for ix in range(-2,2+1):
        for iy in range(-2,2+1):
            if ix*iy<0:
                Q=-3
            else:
                Q=3
            p = [10*ix,10*iy,Q]
            _point.append(p)
    return _point
# make_multi_point()

In [155]:
#---------------------------------------------------------
# Point setting -------------------
# point range -> 
# -100 <=x<=100 , -100<=y<=100
_point=[
    [-50,0,3],
    [50,0, 3],
#     [0,50,-3],
#     [0,-50,-3]
]
# _point = make_multi_point()
#---------------------------------------------------------
# Surface mapping -------------------
# data,title = set_P(_point)
data,title = set_V(_point)
vmax=3*len(_point)
vmin=-vmax

vmax,vmin=3,-3
data = np.where(data>vmax, vmax,data) #異常値処理
data = np.where(data<vmin, vmin,data) #異常値処理

#---------------------------------------------------------
plot_surface(data,vmin=-vmax,vmax=vmax,title=title) #2つの電荷のみ
# plot_surface(data,vmin=-vmax,vmax=vmax,title=title) #2つの電荷のみ

-100 99
99 -100


In [124]:
df = load_data()
plot_surface(df.values,title="E-surface")

TypeError: plot_surface() missing 2 required positional arguments: 'vmin' and 'vmax'